In [61]:
from collections import Counter

import pandas as pd
import torch
from torch.utils.data import WeightedRandomSampler, DataLoader

In [71]:
df = pd.read_csv('../data/bert/train.tsv', sep='\t', header=None)
df.head()

,0,1,2,3,4
0,10582,time in dayton ohio now,"Current Local Time: Dayton, Ohio is in the Eas...",1,0
1,10592,bryan orthopedic surgeon,Dr. Bryan Ming is an orthopedic surgeon in For...,1,0
2,100208,how much is a video editor?,"500-800 a day freelance, mostly reality, in LA...",0,5
3,10546,ticket price for metallica concert,"No matter what you're looking to spend, Vivid ...",1,6
4,100548,can you eat potatoes on a diet,"Potato has about 100 calories in it, so if you...",0,4


In [75]:
train_targets = df.iloc[:,3].values
train_targets

array([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0], dtype=int64)

In [76]:
Counter(train_targets)

Counter({1: 47, 0: 48})

In [81]:
train_targets = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [86]:
count = Counter(train_targets)
class_sample_counts =[0]*len(count)
for k,v in count.items():
    class_sample_counts[k] = v
print(class_sample_counts)

weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
print(weights)

[90, 10]
tensor([0.0111, 0.1000])


In [89]:
samples_weights = weights[train_targets]
samples_weights

tensor([0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111,
        0.1000, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111,
        0.0111, 0.1000, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111,
        0.0111, 0.0111, 0.1000, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111,
        0.0111, 0.0111, 0.0111, 0.1000, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111,
        0.0111, 0.0111, 0.0111, 0.0111, 0.1000, 0.0111, 0.0111, 0.0111, 0.0111,
        0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.1000, 0.0111, 0.0111, 0.0111,
        0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.1000, 0.0111, 0.0111,
        0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.1000, 0.0111,
        0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.1000,
        0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111, 0.0111,
        0.1000])

In [87]:
sampler = WeightedRandomSampler(
    weights=samples_weights,
    num_samples=len(samples_weights),
    replacement=True)

train_data = ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '1',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '2',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '3',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '4',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '5',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '6',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '7',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '8',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '9',
             'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '10']

train_dataloader = DataLoader(train_data, sampler=sampler, batch_size=10)

In [88]:
for s in train_dataloader:
    print(s)

['1', 'x', '6', '3', 'x', 'x', '10', 'x', '3', '8']
['x', '4', '8', '1', 'x', 'x', '6', 'x', '3', '6']
['2', '1', 'x', '6', '5', '7', 'x', 'x', 'x', '6']
['x', '5', '7', 'x', 'x', '6', '1', '2', 'x', 'x']
['x', 'x', '8', 'x', '9', '9', '10', '4', '9', 'x']
['x', 'x', '6', '9', 'x', '1', '6', '2', '6', '8']
['3', 'x', '4', 'x', '5', 'x', 'x', 'x', '9', 'x']
['10', '7', 'x', 'x', 'x', '3', 'x', 'x', 'x', '1']
['2', '5', 'x', '5', 'x', '6', '3', 'x', '3', '3']
['1', '3', '6', '6', '10', 'x', 'x', 'x', '1', 'x']


### Conclusion: RandomWeightSampler oversamples to make training batches balanced